In [1]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv

# 🔹 Last inn API-nøkkel fra .env
load_dotenv()
api_key = os.getenv("API_KEY2")  # Din API-nøkkel
base_url = "https://frost.met.no/observations/v0.jsonld"

# 🔹 Sjekk om API-nøkkel er hentet riktig
if not api_key:
    raise ValueError("❌ API-nøkkel mangler i .env-filen!")

# 🔹 Definer parametere
source_id = "SN68360"  # Trondheim Voll
elements = "air_temperature,wind_speed,air_pressure_at_sea_level,precipitation_amount"
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# 🔹 Bygg URL for forespørselen
url = f"{base_url}?sources={source_id}&elements={elements}&referencetime={start_date}/{end_date}"

# 🔹 Send API-forespørsel med autentisering
response = requests.get(url, auth=(api_key, ""))

# 🔹 Håndter API-responsen
if response.status_code == 200:
    data = response.json()
else:
    raise ValueError(f"❌ Feil ved API-forespørsel: {response.status_code} - {response.text}")

# 🔹 Ekstraher relevant værdata
weather_data = []
for entry in data.get("data", []):
    timestamp = entry.get("referenceTime", "Ukjent tidspunkt")
    source = entry.get("sourceId", "Ukjent stasjon")
    observations = entry.get("observations", [])

    weather_entry = {"Tidspunkt": timestamp, "Værstasjon": source}
    for obs in observations:
        element = obs.get("elementId", "Ukjent element")
        value = obs.get("value", "Ukjent verdi")
        weather_entry[element] = value

    weather_data.append(weather_entry)

# 🔹 Konverter til Pandas DataFrame
df = pd.DataFrame(weather_data)

# 🔹 Lagre til CSV-fil
df.to_csv("trondheim_vaerdata.csv", index=False, encoding="utf-8")

# 🔹 Vis data
print("✅ Data lagret i trondheim_vaerdata.csv")
print(df.head())


ValueError: ❌ Feil ved API-forespørsel: 401 - {
  "@context" : "https://frost.met.no/schema",
  "@type" : "ErrorResponse",
  "apiVersion" : "v0",
  "license" : "https://creativecommons.org/licenses/by/3.0/no/",
  "createdAt" : "2025-03-19T14:52:43Z",
  "queryTime" : 0,
  "currentLink" : "https://frost.met.no/observations/v0.jsonld?sources=SN68360&elements=air_temperature,wind_speed,air_pressure_at_sea_level,precipitation_amount&referencetime=2025-02-17/2025-03-19",
  "error" : {
    "code" : 401,
    "message" : "Unauthorized",
    "reason" : "Invalid basic authentication"
  }
}